In [14]:
import geemap
import ee
import os


In [15]:
#ee.Authenticate()

# Define Functions to Retrive, Display, and Save Images

In [16]:
def RetrieveNAIPimage(lat, long, deltalr = 0.00165, deltaud = 0.00135, date1 = '2018-01-01', date2 = '2020-12-31'):
    region = ee.Geometry.BBox(long- deltalr, lat - deltaud, long + deltalr, lat + deltaud)
    dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date(date1, date2) ).filter(ee.Filter.bounds(region))
    return dataset

def DisplayMap(dataset, lat, long, verbose = True, deltalr = 0.00165, deltaud = 0.00135, zoom = 18):  
    image = dataset.first()
    trueColor = dataset.select(['R', 'G', 'B'])
    trueColorVis = {
      min: 50,
      max: 240,
    }
    if verbose:
        display("images found ",dataset.size())
        print(geemap.image_props(image).getInfo())
        print(geemap.image_stats(image, scale=18).getInfo())
        display(image.geometry().getInfo())
    region = ee.Geometry.BBox(long- deltalr, lat - deltaud, long + deltalr, lat + deltaud)
    regionDebug = ee.Geometry.Polygon(image.getInfo()['properties']['system:footprint']['coordinates'])
    Map = geemap.Map(center=(lat, long), zoom=18)
    Map.addLayer(trueColor, trueColorVis, 'True Color');
    Map.addLayer(region, {'color': 'red'})
    Map.addLayer(regionDebug, {'color': 'green'})
    Map.setOptions('SATELLITE');
    image = dataset.first()

    return Map

def SaveNAIPimage(image ,lat, long, folder, filename, verbose = True, deltalr = 0.00165, deltaud = 0.00135):
    from PIL import Image
    import numpy as np
    fileError.append((folder,filename, lat, long))
    fileTried.append((folder,filename, lat, long, False))
    try:
        region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
        rgb_img = geemap.ee_to_numpy(image, bands=['R', 'G', 'B'], region=region)

        rgb_img[:,2] = rgb_img[:,2] * 2
        rgb_img[:,1] = rgb_img[:,1] * 2
       
        if verbose:
            print(rgb_img.shape)
            print(image.getInfo()['properties']['system:footprint']['coordinates'])
        PIL_image = Image.fromarray(np.uint8(rgb_img)).convert('RGB')
        # Setting the points for cropped image
        left = 9
        top = 0 
        right = 500
        bottom = 500
         
        # Cropped image of above dimension
        # (It will not change original image)
        PIL_image = PIL_image.crop((left, top, right, bottom))
             
        PIL_image.save(f"{folder}/{filename}.png")
        fileError.remove((folder,filename, lat, long))
        fileTried.remove((folder,filename, lat, long, False))
        fileTried.append((folder,filename, lat, long, True))
    except:
        print(f'{(folder,filename, lat, long)}\nThere was a problem:\nLikely the requested image lies outside or overlaps two NAIP images\nOR deltalr and deltaud may be too large for this coordinate - try reducing - the request image is too large' )

In [17]:
# from PIL import Image 
# im = Image.open("Downloads/Morrison/151swe1.png")
# width, height = im.size
# print(width, height)
# # Setting the points for cropped image
# left = 9
# top = 0 
# right = width
# bottom = 500
 
# # Cropped image of above dimension
# # (It will not change original image)
# im1 = im.crop((left, top, right, bottom))
# im1

# Display Map

Red region is area to be saved

In [18]:
import geemap
import ee
import os
# If it is not yet authenticated, uncomment the line below
# ee.Authenticate()

ee.Initialize()

long, lat = -106.922, 35.5123  # MM good
# long, lat = -106.940651, 35.53188472  
# long, lat = -106.940651, 35.53188472 #region overlaps two images
# long, lat = -109.2585029593945, 37.0585735128308 region of interest barley touch an image
# 012bal2.png 35.53188472 -106.940651 Image.sampleRectangle: Fully masked pixels
# 038cary1.png 35.49872169 -106.9102123
# 080mous1.png 35.49908102 -106.8945495
# 085nwnm04.png 36.66582774 -108.9415518
# 087pel1.png 35.49812416 -106.8880206
# 087pel1.png 35.49812416 -106.8880206
# 149spn1.png 35.5207589 -106.9407309
# 158utblan01.png 37.56037407 -109.5675555
# 162utblan05.png 37.57082142 -109.5657423
# 173utmoab3.png 38.73286634 -109.6891729
# 173utmoab3.png 38.73286634 -109.6891729
# 198Untitled Placemark.png 37.06072646136388 -109.1875045313783
# 249Untitled Placemark.png 40.3902717850813 -109.3140870390344 

dataset = RetrieveNAIPimage(lat, long)
Map = DisplayMap(dataset, lat, long, verbose = False, zoom = 18)
Map

Map(center=[35.5123, -106.922], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

# Images from NAIP: Public Domain

https://data.nal.usda.gov/dataset/naip-digital-ortho-photo-image-geospatialdatapresentationform-remote-sensing-image

In [19]:
dataset.getInfo()['properties']['description']

'<p>The National Agriculture Imagery Program (NAIP) acquires aerial imagery\nduring the agricultural growing seasons in the continental U.S.</p><p>NAIP projects are contracted each year based upon available funding and the\nimagery acquisition cycle. Beginning in 2003, NAIP was acquired on\na 5-year cycle. 2008 was a transition year, and a three-year cycle began\nin 2009.</p><p>NAIP imagery is acquired at a one-meter ground sample distance (GSD) with a\nhorizontal accuracy that matches within six meters of photo-identifiable\nground control points, which are used during image inspection.</p><p>Older images were collected using 3 bands (Red, Green, and Blue: RGB), but\nnewer imagery is usually collected with an additional near-infrared band\n(RGBN). RGB asset ids begin with &#39;n<em>&#39;, NRG asset ids begin with &#39;c</em>&#39;, RGBN\nasset ids begin with &#39;m_&#39;.</p><p><b>Provider: <a href="https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip

In [20]:
# Debug
from PIL import Image
import numpy as np

dataset = RetrieveNAIPimage(lat, long)
image = dataset.first()
folder = 'Downloads/NotMorrison'
filename = 'bob'
verbose = False
deltalr = 0.00164
deltaud = 0.00134
region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
rgb_img = geemap.ee_to_numpy(image, bands=['R', 'G', 'B'], region=region)

# region = ee.Geometry.BBox(long - deltalr, lat - deltaud, long + deltalr, lat + deltaud)
# rgb_img = geemap.ee_to_numpy(image, bands=['R', 'G', 'B'], region=region)

# rgb_img[:,2] = rgb_img[:,2] * 2
# rgb_img[:,1] = rgb_img[:,1] * 2

# if verbose:
#     print(rgb_img.shape)
#     print(image.getInfo()['properties']['system:footprint']['coordinates'])
# PIL_image = Image.fromarray(np.uint8(rgb_img)).convert('RGB')
# # Setting the points for cropped image
# left = 9
# top = 0 
# right = 500
# bottom = 500
 
# # Cropped image of above dimension
# # (It will not change original image)
# PIL_image = PIL_image.crop((left, top, right, bottom))
     
# PIL_image.save(f"{folder}/{filename}.png")

# Save Image

Must reduce dataset down to a single image via filtering and perhaps median aggregation


In [21]:
from PIL import Image
import numpy as np
fileError = []
fileTried = []
dataset = RetrieveNAIPimage(lat, long)
image = dataset.first()
folder = 'Downloads/NotMorrison'
filename = 'bob'
SaveNAIPimage(image, lat, long, folder, filename, verbose = False)


# Get Coordinates of Region of interest

In [22]:
#ee.Geometry.Polygon(image.getInfo()['properties']['system:footprint']['coordinates'])
image.getInfo()['properties']['system:footprint']['coordinates']

[[-106.93910995575098, 35.497635459141364],
 [-106.93910914513336, 35.497635413534226],
 [-106.87182644737554, 35.49869837947438],
 [-106.87182347598075, 35.49870059145607],
 [-106.87182014441693, 35.49870253036097],
 [-106.8718199643324, 35.49870336797483],
 [-106.87335906194622, 35.564919058435486],
 [-106.87336185327526, 35.56492149683378],
 [-106.87336419438995, 35.56492424318119],
 [-106.87336500477582, 35.56492437022585],
 [-106.87336581516169, 35.564924416157005],
 [-106.9407038156018, 35.56385882063984],
 [-106.94070678788687, 35.56385660819355],
 [-106.94071012044894, 35.563854668209196],
 [-106.94071030058743, 35.56385383129296],
 [-106.93911590028021, 35.49764069033339],
 [-106.93911310815282, 35.497638251704245],
 [-106.93911076636857, 35.49763550474848],
 [-106.93910995575098, 35.497635459141364]]

In [23]:
import pandas as pd
import time
time.time()
SampleTheseCoords = pd.read_csv('SampleTheseCoords.csv')
arr = SampleTheseCoords.to_numpy()
N = arr.shape[0]
N

410

In [24]:
SampleTheseCoords

,Folder,Filename,Longitude,Latitude
0,Morrison,ant10,-106.852415,35.517027
1,Morrison,ant11,-106.853182,35.520194
2,Morrison,ant12,-106.853518,35.522837
3,Morrison,ant13,-106.853580,35.526295
4,Morrison,ant14,-106.854354,35.529179
...,...,...,...,...
405,NotMorrison,Untitled Placemark,-107.157333,35.407494
406,NotMorrison,Untitled Placemark,-107.156667,35.410396
407,NotMorrison,Untitled Placemark,-107.154161,35.410543
408,NotMorrison,Untitled Placemark,-107.152206,35.411627


# Serial acuisition of images

In [25]:
# from tqdm import tqdm

# start = time.time()
# print(start)
# for i, row in enumerate(tqdm(arr)):
#     folder = f'Downloads/{row[0]}/'
#     filename = f'{i:03}{row[1]}.png'
#     long = row[2]
#     lat = row[3]
#     dataset = RetrieveNAIPimage(lat, long)
#     image = dataset.first()
#     SaveNAIPimage(image ,lat, long, folder, filename, verbose = False)
# end = time.time()
# print(f'Elapsed: {(end-start)/60:5.1f} min')

# Dask Parallel Acquisition of images

In [26]:
from tqdm import tqdm
import dask
lazy  = []
fileError = []
fileTried = []

def kernel(row):
    #print(row)
    folder = row[0]
    filename = row[1]
    long = float(row[2])
    lat = float(row[3])
    dataset = RetrieveNAIPimage(lat, long)
    image = dataset.first()
    SaveNAIPimage(image, lat, long, folder, filename, verbose = False)
    lazy.append(row)
    return row
    
ee.Initialize()
start = time.time()
print(start)
for i in range(0,N):
    row = arr[i]
    row[0] = f'Downloads/{row[0]}/'
    row[1] = f'{i:03}{row[1]}'
    res = dask.delayed(kernel)(row)
    lazy.append(res)
start = time.time()    
dask.compute(*lazy)
end = time.time()
print(f'Elapsed: {(end-start)/60:5.1f} min')

1695436801.4086154
Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'R' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().
('Downloads/Morrison/', '085nwnm04', 36.66582774, -108.9415518)
There was a problem:
Likely the requested image lies outside or overlaps two NAIP images
OR deltalr and deltaud may be too large for this coordinate - try reducing - the request image is too large
Image.sampleRectangle: Fully masked pixels / pixels outside of the image footprint when sampling band 'G' with no default value set. Note that calling sampleRectangle() on an image after ee.Image.clip() may result in a sampling bounding box outside the geometry passed to clip().
('Downloads/Morrison/', '179worth1', 35.48978582, -106.8720177)
There was a problem:
Likely the requested image lies outside or overlaps two NAIP images
OR delt

In [27]:
Files = np.array(fileTried)
Files

array([['Downloads/Morrison/', '085nwnm04', '36.66582774',
        '-108.9415518', 'False'],
       ['Downloads/Morrison/', '179worth1', '35.48978582',
        '-106.8720177', 'False'],
       ['Downloads/NotMorrison/', '278Untitled Placemark', '38.86381872',
        '-120.1215076', 'False'],
       ...,
       ['Downloads/NotMorrison/', '343Untitled Placemark', '35.58493547',
        '-106.7866443', 'True'],
       ['Downloads/NotMorrison/', '393Untitled Placemark', '35.38751185',
        '-107.1500244', 'True'],
       ['Downloads/Morrison/', '144sig5', '35.51722334', '-106.908966',
        'True']], dtype='<U32')

In [28]:
FilesAfterAcquisition = pd.DataFrame(Files, columns=['Folder', 'Filename', 'Latitude', 'Longitude', 'Exists'])

In [29]:
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'True')].to_csv('FilesAfterAcquisition.csv')
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'False')].to_csv('FilesFailedAcquisition.csv')
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'True') & (FilesAfterAcquisition['Folder'] == 'Downloads/NotMorrison/')  ]

,Folder,Filename,Latitude,Longitude,Exists
5,Downloads/NotMorrison/,259Untitled Placemark,41.15713113,-112.4581223,True
6,Downloads/NotMorrison/,199Untitled Placemark,37.05177898,-109.245447,True
7,Downloads/NotMorrison/,381Untitled Placemark,35.10539834,-106.3437911,True
8,Downloads/NotMorrison/,277Untitled Placemark,38.94648755,-120.1741187,True
9,Downloads/NotMorrison/,380Untitled Placemark,35.10293421,-106.3569876,True
...,...,...,...,...,...
402,Downloads/NotMorrison/,404Untitled Placemark,35.40634079,-107.1525276,True
404,Downloads/NotMorrison/,250Untitled Placemark,40.39085022,-109.3417338,True
405,Downloads/NotMorrison/,349Untitled Placemark,35.58862823,-106.8117197,True
407,Downloads/NotMorrison/,343Untitled Placemark,35.58493547,-106.7866443,True


In [30]:
FilesAfterAcquisition[ (FilesAfterAcquisition['Exists'] == 'True') & (FilesAfterAcquisition['Folder'] == 'Downloads/Morrison/')  ]

,Folder,Filename,Latitude,Longitude,Exists
12,Downloads/Morrison/,131sig1,35.52620522,-106.9128848,True
15,Downloads/Morrison/,009ant8,35.50949779,-106.8499463,True
17,Downloads/Morrison/,155ut03,37.01290363,-109.1441284,True
18,Downloads/Morrison/,047chi3,35.50067341,-106.8977549,True
19,Downloads/Morrison/,016bur3,35.51601261,-106.9030999,True
...,...,...,...,...,...
400,Downloads/Morrison/,002ant12,35.52283681,-106.8535179,True
401,Downloads/Morrison/,061fin3,35.53734161,-106.9175565,True
403,Downloads/Morrison/,078mm3,35.50967646,-106.9254147,True
406,Downloads/Morrison/,177utmoab7,38.7322612,-109.6922955,True
